In [ ]:
!pip install pangram-sdk

In [ ]:
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

from sqlalchemy import create_engine, inspect
import os

# build URL from the locally‑forwarded port
user     = os.getenv("DB_USER")
pw       = os.getenv("DB_PASSWORD")
host     = os.getenv("DB_HOST")
port     = os.getenv("DB_PORT")
db       = os.getenv("DB_NAME")
engine   = create_engine(f"postgresql://{user}:{pw}@{host}:{port}/{db}")

In [ ]:
from pangram import Pangram

pg_api = os.getenv("PANGRAM_API")
pangram_client = Pangram(api_key=pg_api)
text = "The quick brown fox jumps over the lazy dog."
result = pangram_client.predict(text)
print(result)

In [ ]:
surveys = pd.read_sql("SELECT * FROM survey_responses;", engine)
surveys.head()

In [ ]:
snapshots = pd.read_sql("SELECT * FROM text_snapshots;", engine)
snapshots.head()

In [ ]:
pid_list = []

with open("pid_for_ai_detection.txt", "r") as fle:
    for line in fle:
        pid_list.append(line.strip())
        
len(pid_list), pid_list[0]

In [ ]:
import json

def get_condition(pid):
    filtered = surveys[surveys["participant_id"] == pid]
    if filtered["condition"].nunique() == 1:
        return filtered.iloc[0]["condition"]
    else:
        return "error"

def get_outline(pid):
    """
    Return submitted outline for provided participant id.
    """
    filtered = snapshots[
        (snapshots["participant_id"] == pid) & 
        (snapshots["stage"] == "outline") &
        (snapshots["type"] == "final")
    ]
    
    if len(filtered) > 1:
        return "error"
    return filtered.iloc[0]["text_content"]

def run_pangram_on_outline(condition):
    if condition == "2":
        return False
    return True
    
with open("pangram_on_outline_r3.txt", "w") as fle:
    for pid in pid_list:
        outline = get_outline(pid)
        cond = get_condition(pid)
        pg_results = pangram_client.predict(outline)
        ai_pred = pg_results["ai_likelihood"]
#         print(f'{pid}\t{len(outline)}\t{cond}\t{to_run}\t{ai_pred}\t{pg_results}')
        fle.write(f'{pid}\t{len(outline)}\t{cond}\t{to_run}\t{ai_pred}\t{json.dumps(pg_results)}\n')

In [ ]:
def get_draft(pid):
    """
    Return submitted draft for provided participant id.
    """
    filtered = snapshots[
        (snapshots["participant_id"] == pid) & 
        (snapshots["stage"] == "draft") &
        (snapshots["type"] == "final")
    ]
    
    if len(filtered) == 0:
        print("error: no drafts returned")
        return "error"
    
    if len(filtered) > 1:
        print(f"error: {len(filtered)} drafts returned")
    
    return filtered.iloc[0]["text_content"]

def run_pangram_on_draft(condition):
    if condition == "3":
        return False
    return True

with open("pangram_on_draft_r3.txt", "w") as fle:
    for pid in pid_list:
        draft = get_draft(pid)
        cond = get_condition(pid)
        to_run = run_pangram_on_draft(cond)
        if draft == "error":
            to_run = False
        pg_results = {}
        ai_pred = "null"
        if to_run:
            pg_results = pangram_client.predict(draft)
            ai_pred = pg_results["ai_likelihood"]
#         print(f'{pid}\t{len(draft)}\t{cond}\t{to_run}\t{ai_pred}\t{pg_results}')
        fle.write(f'{pid}\t{len(draft)}\t{cond}\t{to_run}\t{ai_pred}\t{json.dumps(pg_results)}\n')